In [ ]:
pip install praw twython yahoo_fin 

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import praw
import matplotlib.pyplot as plt
import math
import datetime as dt
import pandas as pd
import numpy as np
import re
import requests

In [ ]:
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
reddit = praw.Reddit(client_id='J81JLuU3YFixYQ',
                    client_secret='wclRNkuNJ9g5gEAZxogosGiOwRh2TA',
                    user_agent='Quick-stocksocialsentiment', 
                    check_for_async=False)

In [ ]:
df = []

for post in reddit.subreddit('wallstreetbets').hot(limit=500):
    
    content = {
        "title" : post.title,
        "text" : post.selftext
    }
    
    df.append(content)

df = pd.DataFrame(df)

In [ ]:
regex = re.compile('[^a-zA-Z ]')
word_dict = {}

for (index, row) in df.iterrows():
    # titles
    title = row['title']
    
    title = regex.sub('', title)
    title_words = title.split(' ')
    
    # content
    content = row['text']
    
    content = regex.sub('', content)
    content_words = content.split(' ')
    
    # combine
    words = title_words + content_words
    
    for x in words:
        
        if x in ['A', 'B', 'GO', 'ARE', 'ON', 'IT', 'ALL', 'NEXT', 'PUMP', 'AT', 'NOW', 'FOR', 'TD', 'CEO', 'AM', 'K', 'BIG', 'BY', 'LOVE', 'CAN', 'BE', 'SO', 'OUT', 'STAY', 'OR', 'NEW','RH','EDIT','ONE','ANY']:
            pass
        elif x in word_dict:
            word_dict[x] += 1
        else:
            word_dict[x] = 1

word_df = pd.DataFrame.from_dict(list(word_dict.items())).rename(columns = {0:"Term", 1:"Frequency"})

In [ ]:
word_df

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1fb7hfgFD7ZeCMrfNOG0YD83TlIyVNWiGtX7S0m4Dybg/edit#gid=764117698')

In [ ]:
sheet = wb.worksheet('tickers')

In [ ]:
data = sheet.get_all_values()

In [ ]:
df = pd.DataFrame(data)
df.columns = df.iloc[0]

In [ ]:
ticker_df = df.iloc[1:].rename(columns = {"Symbol":"Term", "Name":"Company_Name"})
ticker_df

,Term,Company_Name
1,A,Agilent Technologies Inc. Common Stock
2,AA,Alcoa Corporation Common Stock
3,AACG,ATA Creativity Global American Depositary Shares
4,AACQ,Artius Acquisition Inc. Class A Common Stock
5,AACQU,Artius Acquisition Inc. Unit
...,...,...
7267,ZVO,Zovio Inc. Common Stock
7268,ZYME,Zymeworks Inc. Common Shares
7269,ZYNE,Zynerba Pharmaceuticals Inc. Common Stock
7270,ZYXI,Zynex Inc. Common Stock


In [ ]:
stonks_df = pd.merge(ticker_df, word_df, on="Term")

In [ ]:
stonks_df = stonks_df.sort_values(by="Frequency", ascending = False, ignore_index = True).head(20)

In [ ]:
output = "\n\n~~Top 10 Stonks on WSB~~\n"+ "1) "+ stonks_df['Company_Name'][0] + " (" + stonks_df['Term'][0] + ") " + " - " + str(stonks_df['Frequency'][0]) + " mentions\n"+ "2) "+ stonks_df['Company_Name'][1] + " (" + stonks_df['Term'][1] + ") " + " - " + str(stonks_df['Frequency'][1]) + " mentions\n"+ "3) "+ stonks_df['Company_Name'][2] + " (" + stonks_df['Term'][2] + ") " + " - " + str(stonks_df['Frequency'][2]) + " mentions\n"+ "4) "+ stonks_df['Company_Name'][3] + " (" + stonks_df['Term'][3] + ") " + " - " + str(stonks_df['Frequency'][3]) + " mentions\n"+ "5) "+ stonks_df['Company_Name'][4] + " (" + stonks_df['Term'][4] + ") " + " - " + str(stonks_df['Frequency'][4]) + " mentions\n"+ "6) "+ stonks_df['Company_Name'][5] + " (" + stonks_df['Term'][5] + ") " + " - " + str(stonks_df['Frequency'][5]) + " mentions\n"+ "7) "+ stonks_df['Company_Name'][6] + " (" + stonks_df['Term'][6] + ") " + " - " + str(stonks_df['Frequency'][6]) + " mentions\n"+ "8) "+ stonks_df['Company_Name'][7] + " (" + stonks_df['Term'][7] + ") " + " - " + str(stonks_df['Frequency'][7]) + " mentions\n"+ "9) "+ stonks_df['Company_Name'][8] + " (" + stonks_df['Term'][8] + ") " + " - " + str(stonks_df['Frequency'][8]) + " mentions\n"+ "10) "+ stonks_df['Company_Name'][9] + " (" + stonks_df['Term'][9] + ") " + " - " + str(stonks_df['Frequency'][9]) + " mentions"
print (output)



~~Top 10 Stonks on WSB~~
1) Cleveland-Cliffs Inc. Common Stock (CLF)  - 142 mentions
2) Clover Health Investments Corp. Class A Common Stock (CLOV)  - 130 mentions
3) ContextLogic Inc. Class A Common Stock (WISH)  - 108 mentions
4) DuPont de Nemours Inc. Common Stock (DD)  - 74 mentions
5) Clean Energy Fuels Corp. Common Stock (CLNE)  - 74 mentions
6) BlackBerry Limited Common Stock (BB)  - 69 mentions
7) AMC Entertainment Holdings Inc. Class A Common Stock (AMC)  - 66 mentions
8) UWM Holdings Corporation Class A Common Stock (UWMC)  - 58 mentions
9) GameStop Corporation Common Stock (GME)  - 51 mentions
10) Workhorse Group Inc. Common Stock (WKHS)  - 40 mentions


In [ ]:
stonk_list = stonks_df['Term'].to_numpy()
stonk_list

array(['CLF', 'CLOV', 'WISH', 'DD', 'CLNE', 'BB', 'AMC', 'UWMC', 'GME',
       'WKHS', 'M', 'EV', 'MP', 'TLRY', 'WWE', 'SI', 'RKT', 'SNDL', 'C',
       'SENS'], dtype=object)

In [ ]:
sub_reddits = reddit.subreddit('wallstreetbets')
stocks = stonk_list 

In [ ]:
def commentSentiment(ticker, urlT):
    subComments = []
    bodyComment = []
    try:
        check = reddit.submission(url=urlT)
        subComments = check.comments
    except:
        return 0
    
    for comment in subComments:
        try: 
            bodyComment.append(comment.body)
        except:
            return 0
    
    sia = SIA()
    results = []
    for line in bodyComment:
        scores = sia.polarity_scores(line)
        scores['headline'] = line

        results.append(scores)
    
    df =pd.DataFrame.from_records(results)
    df.head()
    df['label'] = 0
    
    try:
        df.loc[df['compound'] > 0.1, 'label'] = 1
        df.loc[df['compound'] < -0.1, 'label'] = -1
    except:
        return 0
    
    averageScore = 0
    position = 0
    while position < len(df.label)-1:
        averageScore = averageScore + df.label[position]
        position += 1
    averageScore = averageScore/len(df.label) 
    
    return(averageScore)

In [ ]:
def latestComment(ticker, urlT):
    subComments = []
    updateDates = []
    try:
        check = reddit.submission(url=urlT)
        subComments = check.comments
    except:
        return 0
    
    for comment in subComments:
        try: 
            updateDates.append(comment.created_utc)
        except:
            return 0
    
    updateDates.sort()
    return(updateDates[-1])

In [ ]:
def get_date(date):
    return dt.datetime.fromtimestamp(date)

In [ ]:
submission_statistics = []
d = {}
for ticker in stocks:
    for submission in reddit.subreddit('wallstreetbets').search(ticker, limit=100):
        if submission.domain != "self.wallstreetbets":
            continue
        d = {}
        d['ticker'] = ticker
        d['num_comments'] = submission.num_comments
        d['comment_sentiment_average'] = commentSentiment(ticker, submission.url)
        if d['comment_sentiment_average'] == 0.000000:
            continue
        d['latest_comment_date'] = latestComment(ticker, submission.url)
        d['score'] = submission.score
        d['upvote_ratio'] = submission.upvote_ratio
        d['date'] = submission.created_utc
        d['domain'] = submission.domain
        d['num_crossposts'] = submission.num_crossposts
        d['author'] = submission.author
        submission_statistics.append(d)
    
dfSentimentStocks = pd.DataFrame(submission_statistics)

_timestampcreated = dfSentimentStocks["date"].apply(get_date)
dfSentimentStocks = dfSentimentStocks.assign(timestamp = _timestampcreated)

_timestampcomment = dfSentimentStocks["latest_comment_date"].apply(get_date)
dfSentimentStocks = dfSentimentStocks.assign(commentdate = _timestampcomment)

dfSentimentStocks.sort_values("latest_comment_date", axis = 0, ascending = True,inplace = True, na_position ='last') 

In [ ]:
dfSentimentStocks.author.value_counts()

bigbear0083            25
jjd1226                10
n8hckns                 7
pennyether              6
CommunicationIll570     5
                       ..
nitsujsc                1
BrotherLuminous         1
Flying_madman           1
SorryItsBrexit          1
Ummmmmmmmmmm_           1
Name: author, Length: 573, dtype: int64

In [ ]:
dfSentimentStocks

,ticker,num_comments,comment_sentiment_average,latest_comment_date,score,upvote_ratio,date,domain,num_crossposts,author,timestamp,commentdate
573,WWE,8,0.600000,1.394244e+09,5,0.78,1.394172e+09,self.wallstreetbets,0,pedal_bike,2014-03-07 05:58:30,2014-03-08 01:54:15
572,WWE,12,0.200000,1.410917e+09,0,0.50,1.410854e+09,self.wallstreetbets,0,mattysmitty,2014-09-16 07:46:09,2014-09-17 01:18:32
571,WWE,4,0.500000,1.460419e+09,16,1.00,1.460405e+09,self.wallstreetbets,0,TheFadedBull,2016-04-11 20:10:45,2016-04-11 23:55:42
570,WWE,7,-0.333333,1.460499e+09,4,0.67,1.460496e+09,self.wallstreetbets,0,Jeffde,2016-04-12 21:21:35,2016-04-12 22:02:00
789,SENS,193,0.382353,1.470173e+09,605,0.97,1.461691e+09,self.wallstreetbets,0,TheCudgeler,2016-04-26 17:18:08,2016-08-02 21:20:01
...,...,...,...,...,...,...,...,...,...,...,...,...
470,TLRY,126,0.238095,1.623336e+09,220,0.81,1.623315e+09,self.wallstreetbets,2,Jaded-Offer-2816,2021-06-10 08:42:11,2021-06-10 14:46:42
518,WWE,2,-0.500000,1.623337e+09,13,0.78,1.623336e+09,self.wallstreetbets,0,aphelion3342,2021-06-10 14:42:05,2021-06-10 14:48:26
517,WWE,38,0.071429,1.623337e+09,157,0.71,1.623331e+09,self.wallstreetbets,0,pennyether,2021-06-10 13:24:03,2021-06-10 14:48:43
617,SI,38,0.071429,1.623337e+09,196,0.74,1.623331e+09,self.wallstreetbets,0,pennyether,2021-06-10 13:24:03,2021-06-10 14:48:43


In [ ]:
from google.colab import drive
drive.mount('/drive')
dfSentimentStocks.to_excel('/drive/My Drive/SGUS Fintech Algo-trading BI Project/Working Folders/Phase 3 Integration Algo-T + BI/JW/Reddit_Sentiment_Score.xlsx')

Mounted at /drive
